# **Assignment 4**
## **Group 3:**
* Karyl Grasparil
* Mariah Quinquito
* Yanna Polonia

In [61]:
import gensim.downloader as api
import nltk
import re
from autocorrect import Speller
from gensim.models import KeyedVectors, Word2Vec
from nltk.corpus import gutenberg
from nltk.stem import WordNetLemmatizer 
from nltk import tokenize
from scipy.spatial.distance import cosine

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# **1. Data Steps**

## Using nltk.corpus.gutenberg.raw to load the entire text of plays: 1) The Tragedy of Hamlet, Prince of Denmark, 2) The Tragedy of Macbeth, and 3) The Tragedy of Julius Caesar into a single variable

In [2]:
# Downloading Gutenberg corpus
nltk.download('gutenberg')

# Exploring file identifiers included in the Gutenberg corpus
nltk.corpus.gutenberg.fileids()

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/ypolonia/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
# Adding each file name into the variable plays_raw
plays_raw = gutenberg.raw('shakespeare-hamlet.txt') + gutenberg.raw('shakespeare-macbeth.txt') + gutenberg.raw('shakespeare-caesar.txt')

# Printing the last 500 characters in plays_raw
print(plays_raw[-500:])

rall honest thought,
And common good to all, made one of them.
His life was gentle, and the Elements
So mixt in him, that Nature might stand vp,
And say to all the world; This was a man

   Octa. According to his Vertue, let vs vse him
Withall Respect, and Rites of Buriall.
Within my Tent his bones to night shall ly,
Most like a Souldier ordered Honourably:
So call the Field to rest, and let's away,
To part the glories of this happy day.

Exeunt. omnes.


FINIS. THE TRAGEDIE OF IVLIVS CaeSAR.





In [4]:
## Lowering the case of play_raw content
plays_raw = plays_raw.lower()

# Printing the first 100 characters to validate case changes made
plays_raw[:100]

'[the tragedie of hamlet by william shakespeare 1599]\n\n\nactus primus. scoena prima.\n\nenter barnardo a'

## Using Speller from the autocorrect library to correct spelling mistakes

In [5]:
speller = Speller(lang = 'en')

In [6]:
plays_corrected = speller(plays_raw)

In [7]:
plays_corrected[-500:]

"a general honest thought,\nand common good to all, made one of them.\nhis life was gentle, and the elements\nso mix in him, that nature might stand vp,\nand say to all the world; this was a man\n\n   oct. according to his virtue, let vs vse him\nwithall respect, and rites of burial.\nwithin my tent his bones to night shall ly,\nmost like a soldier ordered honourable:\nso call the field to rest, and let's away,\nto part the stories of this happy day.\n\nevent. ones.\n\n\nfinish. the tragedies of ivlivs cesar.\n\n\n"

## Tokenizing the text into sentences, and then each sentence into words.

In [8]:
# Downloading the Punkt tokenizer module to ease text splitting into sentences and words, and be able to handle tab-separated values.
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ypolonia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
# Storing a sentence-tokenized copy of plays_raw in sentences_text
sentences_txt = tokenize.sent_tokenize(plays_raw)

# Verifying sentences_txt data type and its length 
type(sentences_txt), len(sentences_txt)

(list, 5325)

In [10]:
# Printing the first 10 sentences
sentences_txt[:10]

['[the tragedie of hamlet by william shakespeare 1599]\n\n\nactus primus.',
 'scoena prima.',
 'enter barnardo and francisco two centinels.',
 'barnardo.',
 "who's there?",
 'fran.',
 'nay answer me: stand & vnfold\nyour selfe\n\n   bar.',
 'long liue the king\n\n   fran.',
 'barnardo?',
 'bar.']

In [11]:
# Storing the tokenized words in each sentence as list named word_text
words_txt = [tokenize.word_tokenize(sent) for sent in sentences_txt]

# Verifying words_txt list elements data type and the length of its first element
type(words_txt[0]), len(words_txt[0])

(list, 13)

In [12]:
# Printing the words in the first sentence
words_txt[0]

['[',
 'the',
 'tragedie',
 'of',
 'hamlet',
 'by',
 'william',
 'shakespeare',
 '1599',
 ']',
 'actus',
 'primus',
 '.']

## Using regular expressions (the re library) to do some cleanup of the text

In [13]:
def clean_text(word):
    """
    Function used to clean one word at a time
    """
    
    # Removing words with one to two characters
    word = re.sub('\\b\w{1,2}\\b', '', word)

    # Remove punctuation using regex, keeping only word characters and whitespace
    word = re.sub(r'[^\w]', '', word)
    
    # Removing numbers, so we are just removing numbers that are three digits or more thanks to previous part
    word = re.sub("\d", "", word)
    
    return word

In [14]:
cleaned_sentences = []
cleaned_words = []

for sentence in sentences_txt:
    # Splitting each setence into string elements
    words_sentences = sentence.split()

    # Applying the clean_text function to every splitted setence
    cleaned_sentence = [clean_text(word) for word in words_sentences]
    
    # Removing empty strings in each setence if any
    cleaned_sentence = [item for item in cleaned_sentence if item]

    # Appending each sentence to cleaned_sentences
    cleaned_sentences.append(cleaned_sentence)

    # Extracting from each setence (sublist) the words to append flatten list named cleaned_words
    cleaned_words = [item for sublist in cleaned_sentences for item in sublist]

In [15]:
# Printing the first 5 cleaned words
cleaned_words[:5]

['the', 'tragedie', 'hamlet', 'william', 'shakespeare']

In [16]:
# Printing the first 2 cleaned setences
cleaned_sentences[:2]

[['the', 'tragedie', 'hamlet', 'william', 'shakespeare', 'actus', 'primus'],
 ['scoena', 'prima']]

## Creating a list of stopwords (using publicly available lists and/or adding your own) and removing these.

In [17]:
# Using stop words from https://github.com/stopwords-iso/stopwords-en/blob/master/stopwords-en.txt, which are three or more characters
modern_eng_stop = ["'ll", "'tis", "'twas", "'ve", "a's", "able", "ableabout", "about", "above", "abroad", "abst", "accordance", "according",\
             "accordingly", "across", "act", "actually", "added", "adj", "adopted", "affected", "affecting", "affects", \
             "after", "afterwards", "again", "against", "ago", "ahead", "ain't", "aint", "all", "allow", "allows", \
             "almost", "alone", "along", "alongside", "already", "also", "although", "always", "amid", "amidst", "among", "amongst", \
             "amoungst", "amount", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", \
             "anyway", "anyways", "anywhere", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", \
             "are", "area", "areas", "aren", "aren't", "arent", "arise", "around", "arpa", "aside", "ask", "asked", "asking", \
             "asks", "associated", "at", "au", "auth", "available", "aw", "away", "awfully", "az", "b", "ba", "back", "backed", "backing", \
             "backs", "backward", "backwards", "became", "because", "become", "becomes", "becoming", "been", "before", \
             "beforehand", "began", "begin", "beginning", "beginnings", "begins", "behind", "being", "beings", "believe", "below", \
             "beside", "besides", "best", "better", "between", "beyond", "big", "bill", "billion", "biol", "both", "bottom", "brief", \
             "briefly", "but", "buy", "c'mon", "c's", "call", "came", "can", "can't", "cannot", "cant", "caption", "case", "cases", \
             "cause", "causes", "certain", "certainly", "changes", "clear", "clearly", "click", "cmon", "co.", "com", "come", "comes", \
             "computer", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "copy", \
             "corresponding", "could", "could've", "couldn", "couldn't", "couldnt", "course", "cry", "currently", "dare", "daren't", \
             "darent", "date", "dear", "definitely", "describe", "described", "despite", "detail", "did", "didn", "didn't", "didnt", \
             "differ", "different", "differently", "directly", "does", "doesn", "doesn't", "doesnt", "doing", "don", "don't", "done", \
             "dont", "doubtful", "down", "downed", "downing", "downs", "downwards", "due", "during", "each", "early", "edu", "effect", \
             "eight", "eighty", "either", "eleven", "else", "elsewhere", "empty", "end", "ended", "ending", "ends", "enough", "entirely", \
             "especially", "et-al", "etc", "even", "evenly", "ever", "evermore", "every", "everybody", "everyone", "everything", \
             "everywhere", "exactly", "example", "except", "face", "faces", "fact", "facts", "fairly", "far", "farther", "felt", "few", \
             "fewer", "fifteen", "fifth", "fifty", "fify", "fill", "find", "finds", "fire", "first", "five", "fix", "followed", \
             "following", "follows", "for", "forever", "former", "formerly", "forth", "forty", "forward", "found", "four", "free", \
             "from", "front", "full", "fully", "further", "furthered", "furthering", "furthermore", "furthers", "gave", "general", \
             "generally", "get", "gets", "getting", "give", "given", "gives", "giving", "gmt", "goes", "going", "gone", "good", "goods", \
             "got", "gotten", "gov", "great", "greater", "greatest", "greetings", "group", "grouped", "grouping", "groups", "had", \
             "hadn't", "hadnt", "half", "happens", "hardly", "has", "hasn", "hasn't", "hasnt", "have", "haven", "haven't", "havent", \
             "having", "he'd", "he'll", "he's", "hed", "hell", "hello", "help", "hence", "her", "here", "here's", "hereafter", "hereby", \
             "herein", "heres", "hereupon", "hers", "herself", "herse”", "hes", "hid", "high", "higher", "highest", "him", "himself", \
             "himse”", "his", "hither", "home", "homepage", "hopefully", "how", "how'd", "how'll", "how's", "howbeit", "however", "htm", \
             "html", "http", "hundred", "i'd", "i'll", "i'm", "i've", "i.e.", "ignored", "ill", "immediate", "immediately", "importance", \
             "important", "inasmuch", "inc", "inc.", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", \
             "inside", "insofar", "instead", "int", "interest", "interested", "interesting", "interests", "into", "invention", "inward", \
             "isn", "isn't", "isnt", "it'd", "it'll", "it's", "itd", "itll", "its", "itself", "itse”", "ive", "join", "just", "keep", \
             "keeps", "kept", "keys", "kind", "knew", "know", "known", "knows", "large", "largely", "last", "lately", "later", "latest", \
             "latter", "latterly", "least", "length", "less", "lest", "let", "let's", "lets", "like", "liked", "likely", "likewise", \
             "line", "little", "long", "longer", "longest", "look", "looking", "looks", "low", "lower", "ltd", "made", "mainly", "make", \
             "makes", "making", "man", "many", "may", "maybe", "mayn't", "maynt", "mean", "means", "meantime", "meanwhile", "member", \
             "members", "men", "merely", "microsoft", "might", "might've", "mightn't", "mightnt", "mil", "mill", "million", "mine", \
             "minus", "miss", "more", "moreover", "most", "mostly", "move", "mrs", "msie", "much", "mug", "must", "must've", "mustn't", \
             "mustnt", "myself", "myse”", "name", "namely", "nay", "near", "nearly", "necessarily", "necessary", "need", "needed", \
             "needing", "needn't", "neednt", "needs", "neither", "net", "netscape", "never", "neverf", "neverless", "nevertheless", \
             "new", "newer", "newest", "next", "nine", "ninety", "no-one", "nobody", "non", "none", "nonetheless", "noone", "nor", \
             "normally", "nos", "not", "noted", "nothing", "notwithstanding", "novel", "now", "nowhere", "null", "number", "numbers", \
             "obtain", "obtained", "obviously", "off", "often", "okay", "old", "older", "oldest", "omitted", "once", "one", "one's", \
             "ones", "only", "onto", "open", "opened", "opening", "opens", "opposite", "ord", "order", "ordered", "ordering", "orders", \
             "org", "other", "others", "otherwise", "ought", "oughtn't", "oughtnt", "our", "ours", "ourselves", "out", "outside", "over", \
             "overall", "owing", "own", "page", "pages", "part", "parted", "particular", "particularly", "parting", "parts", "past",\
             "per", "perhaps", "place", "placed", "places", "please", "plus", "pmid", "point", "pointed", "pointing", "points", "poorly", \
             "possible", "possibly", "potentially", "predominantly", "present", "presented", "presenting", "presents", "presumably", \
             "previously", "primarily", "probably", "problem", "problems", "promptly", "proud", "provided", "provides", "put", "puts", \
             "que", "quickly", "quite", "ran", "rather", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", \
             "regarding", "regardless", "regards", "related", "relatively", "research", "reserved", "respectively", "resulted", \
             "resulting", "results", "right", "ring", "room", "rooms", "round", "run", "said", "same", "saw", "say", "saying", "says", \
             "sec", "second", "secondly", "seconds", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "sees", \
             "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "seventy", "several", "shall", "shan't", "shant", \
             "she", "she'd", "she'll", "she's", "shed", "shell", "shes", "should", "should've", "shouldn", "shouldn't", "shouldnt", "show", \
             "showed", "showing", "shown", "showns", "shows", "side", "sides", "significant", "significantly", "similar", "similarly", \
             "since", "sincere", "site", "six", "sixty", "slightly", "small", "smaller", "smallest", "some", "somebody", "someday", \
             "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", \
             "specifically", "specified", "specify", "specifying", "state", "states", "still", "stop", "strongly", "sub", "substantially", \
             "successfully", "such", "sufficiently", "suggest", "sup", "sure", "system", "t's", "take", "taken", "taking", "tell", "ten", \
             "tends", "test", "text", "than", "thank", "thanks", "thanx", "that", "that'll", "that's", "that've", "thatll", "thats", \
             "thatve", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "there'd", "there'll", "there're", \
             "there's", "there've", "thereafter", "thereby", "thered", "therefore", "therein", "therell", "thereof", "therere", "theres", \
             "thereto", "thereupon", "thereve", "these", "they", "they'd", "they'll", "they're", "they've", "theyd", "theyll", "theyre", \
             "theyve", "thick", "thin", "thing", "things", "think", "thinks", "third", "thirty", "this", "thorough", "thoroughly", \
             "those", "thou", "though", "thoughh", "thought", "thoughts", "thousand", "three", "throug", "through", "throughout", "thru", \
             "thus", "til", "till", "tip", "tis", "today", "together", "too", "took", "top", "toward", "towards", "tried", "tries", \
             "trillion", "truly", "try", "trying", "turn", "turned", "turning", "turns", "twas", "twelve", "twenty", "twice", "two", \
             "under", "underneath", "undoing", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "upon", "ups", \
             "upwards", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "uucp", "value", "various", \
             "versus", "very", "via", "viz", "vol", "vols", "want", "wanted", "wanting", "wants", "was", "wasn", "wasn't", "wasnt", "way", \
             "ways", "we'd", "we'll", "we're", "we've", "web", "webpage", "website", "wed", "welcome", "well", "wells", "went", \
             "were", "weren", "weren't", "werent", "weve", "wf", "what", "what'd", "what'll", "what's", "what've", "whatever", \
             "whatll", "whats", "whatve", "when", "when'd", "when'll", "when's", "whence", "whenever", "where", "where'd", "where'll", \
             "where's", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which", \
             "whichever", "while", "whilst", "whim", "whither", "who", "who'd", "who'll", "who's", "whod", "whoever", "whole", \
             "wholl", "whom", "whomever", "whos", "whose", "why", "why'd", "why'll", "why's", "widely", "width", "will", "willing", \
             "wish", "with", "within", "without", "won", "won't", "wonder", "wont", "words", "work", "worked", "working", "works", \
             "world", "would", "would've", "wouldn", "wouldn't", "wouldnt", "www", "year", "years", "yes", "yet", "you", "you'd", \
             "you'll", "you're", "you've", "youd", "youll", "young", "younger", "youngest", "your", "youre", "yours", "yourself", \
             "yourselves", "youve", "zero"]

In [18]:
# Old English stop words provided by ChatGPT (those which are 3+ characters are the considered ones): þe - the", "þæt - that", "þes - this",
# "þa - those", "ne - not", "ac - but", "gif - if", "sē - the (that)", "þū - you", "ic - I", "eall - all", "nān - none", "þīn - your", 
# "wē - we", "ge - you (plural)", "hē - he", "hēo - she", "swā - so", "mid - with", "fram - from", "þurh - through", "tō - to", "þæt - that",
# "hwæðer - whether", "līc - body", "witan - to know", "secgan - to say", "cunnan - to be able", "don - to do", "bēon - to be", 
# "witan - to know", "hæbban - to have", "þearf - need", "hī - they", "þeah - although", "þa - the (those)", "sīe - may be", "wæs - was", 
# "eom - am", "hæfde - had", "ġē - you (plural)", "þa - then", "þæt - that (it)", "būtan - without", "eallunga - altogether", 
# "þū - thou (you)", "nǣfre - never", "hwæt - what", "hwī - why", "þæt - that", "læt - let", "wite - to know", "wæron - were", "swa - such",
# "æfre - ever", "hū - how", "wyrcan - to work", "þinc - to think", "gesīen - to see", "cūðe - known", "hæle - to help", "beon - to be", 
# "sēo - she (the)", "cū - cow (to know)", "dǣl - part", "feorr - far", "mē - me", "bēo - be", "gā - go", "ðā - then", "swīþe - very, 
# "thou - you", "art - are"
old_eng_stop = ["þæt", "þes", "gif", "eall", "nān", "þīn", "hēo", "swā", "mid", "fram", "þurh", "þæt", "hwæðer", "līc", "witan", \
                "secgan", "cunnan", "don", "bēon", "witan", "hæbban", "þearf", "þeah", "sīe", "wæs", "eom", "hæfde", "þæt", "būtan", \
                "eallunga", "nǣfre", "hwæt", "hwī", "þæt", "læt", "wite", "wæron", "swa", "æfre", "wyrcan", "þinc", "gesīen", "cūðe", \
                "hæle", "beon", "sēo", "dǣl", "feorr", "bēo", "swīþe", "thou", "art"]

In [19]:
# using latin stop words from https://www.perseus.tufts.edu/hopper/stopwords which are three or more characters
latin_stop = ["adhic", "aliqui", "aliquis", "ante", "apud", "atque", "aut", "autem", "cum", "cur", "deinde", "dum", "ego", "enim", "ergo", \
              "est", "etiam", "etsi", "fio", "haud", "hic", "iam", "idem", "igitur", "ille", "infra", "inter", "interim", "ipse", "ita", \
              "magis", "modo", "mox", "nam", "nec", "necque", "neque", "nisi", "non", "nos", "per", "possum", "post", "pro", "quae", \
              "quam", "quare", "qui", "quia", "quicumque", "quidem", "quilibet", "quis", "quisnam", "quisquam", "quisque", "quisquis", \
              "quo", "quoniam", "sed", "sic", "sive", "sub", "sui", "sum", "super", "suus", "tam", "tamen", "trans", "tum", "ubi", "uel", \
              "uero", "unus"]

In [20]:
# Adding stop_punct and stopwords
stop_final = modern_eng_stop + old_eng_stop + latin_stop

In [21]:
def drop_stopwords(input_tokens):
    """Removes stop words found in sentence received"""
    return [token for token in input_tokens if token not in stop_final]

In [22]:
# Removing stop words in all of the sentences in cleaned_sentences
sentences_nostop = [drop_stopwords(sentence) for sentence in cleaned_sentences]

# Removing stop words from cleaned_words list
words_nostop = [item for item in cleaned_words if item not in stop_final]

In [23]:
# Printing the first ten words in words_nostop
words_nostop[:10]

['tragedie',
 'hamlet',
 'william',
 'shakespeare',
 'actus',
 'primus',
 'scoena',
 'prima',
 'enter',
 'barnardo']

In [24]:
# Printing the first 5 sentences in words_sentences_nostop
sentences_nostop[:5]

[['tragedie', 'hamlet', 'william', 'shakespeare', 'actus', 'primus'],
 ['scoena', 'prima'],
 ['enter', 'barnardo', 'francisco', 'centinels'],
 ['barnardo'],
 []]

## Using PorterStemmer or WordNetLemmatizer from nltk.stem on the text

In [25]:
# Downloading wordnet in order to use the lemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ypolonia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
# Initializing the lemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatizer(input_token):
    return [lemmatizer.lemmatize(token) for token in input_token]

In [27]:
# Lemmatizing process for words_sentences_nostop
lemmatized_sentences = [perform_lemmatizer(sentence) for sentence in sentences_nostop]

# Lemmatizing process for words_nostop
lemmatized_words = [lemmatizer.lemmatize(item) for item in words_nostop]

In [28]:
# Printing the first lemmatized sentence
lemmatized_sentences[0]

['tragedie', 'hamlet', 'william', 'shakespeare', 'actus', 'primus']

In [29]:
# Printing the first 20 lemmatized words
lemmatized_words[:20]

['tragedie',
 'hamlet',
 'william',
 'shakespeare',
 'actus',
 'primus',
 'scoena',
 'prima',
 'enter',
 'barnardo',
 'francisco',
 'centinels',
 'barnardo',
 'fran',
 'answer',
 'stand',
 'vnfold',
 'selfe',
 'bar',
 'liue']

## <span style='color:blue'>Using lemmatization instead of Stemming because unlike stemming, lemmatization considers the context of the word and pursues the goal of returning a meaningful base form of each word.</span>

## Printing out the words in the first five sentences of the processed text data, to get some ideas of previous steps

In [30]:
lemmatized_sentences[:5]

[['tragedie', 'hamlet', 'william', 'shakespeare', 'actus', 'primus'],
 ['scoena', 'prima'],
 ['enter', 'barnardo', 'francisco', 'centinels'],
 ['barnardo'],
 []]

# 2. Modeling 

## Creating a CBOW word2vec model from gensim.model. Making choices of vector_size, epochs, window, min_count, and possibly other hyperparameters. 

In [31]:
# Creating a CBOW Word2Vec model, using sg = 0 considering CBOW is required
cbow_model = Word2Vec(lemmatized_sentences, vector_size = 100, window = 5, min_count = 2, sg = 0, epochs = 10)

cbow_model.save('cbow_model')

## Training the model on the cleaned Shakespeare text data. Use gensim.model.wv.key_to_index and gensim.model.wv.get_vecattr to print out a list of the 20 most frequent words in the vocabulary along with the word count.

In [32]:
# Getting the 20 most frequent words
most_frequent_words = list(cbow_model.wv.key_to_index.items())[:20]

# Printing the words along with their counts
print("Most Frequent Words and Their Counts:\n")
for word, index in most_frequent_words:
    count = cbow_model.wv.get_vecattr(word, "count")
    print(f"Word: '{word}', Count: {count}")

Most Frequent Words and Their Counts:

Word: 'haue', Count: 448
Word: 'ham', Count: 337
Word: 'lord', Count: 306
Word: 'king', Count: 248
Word: 'caesar', Count: 231
Word: 'enter', Count: 230
Word: 'thy', Count: 202
Word: 'thee', Count: 174
Word: 'brutus', Count: 162
Word: 'vpon', Count: 162
Word: 'bru', Count: 153
Word: 'hath', Count: 144
Word: 'selfe', Count: 138
Word: 'macb', Count: 137
Word: 'heere', Count: 135
Word: 'time', Count: 132
Word: 'sir', Count: 121
Word: 'night', Count: 119
Word: 'speake', Count: 119
Word: 'giue', Count: 118


## Creating a skipgram word2vec model from gensim.model, making choices of vector_size, epochs, window, min_count, and possibly other hyperparameters and training the model on the cleaned Shakespeare text data 

In [33]:
# Creating a skipgram Word2Vec model, using sg = 1 considering skipgram is required
skipgram_model = Word2Vec(lemmatized_sentences, vector_size = 100, window = 5, min_count = 2, sg = 1, epochs = 10)

skipgram_model.save('skipgram_model')

## Loading the pretrained GloVe model from gensim.models.keyedvectors for comparison with the models trained on Shakespeare text

In [34]:
# Loading GloVe file
glove_file = 'glove.6B.100d.txt'

# Loading the GloVe model
glove_model = KeyedVectors.load_word2vec_format(glove_file, binary = False, no_header = True)

glove_model.save('glove_model')

### <span style='color:blue'>The glove.6B.100d.txt file contains pre-trained word embeddings from the GloVe (Global Vectors for Word Representation) model. Considering the vector size parameter choose for previous models, we are selecting this particular glove file, so there is a correspondence between the input data of each model (data is captured in a 100-dimensional space). In this glove file, each line in the file corresponds to a single word and its vector representation (the file contains vectors for 400,000 words and phrases).</span>

# 3. Discussion

## Comparing the three models by finding the 5 most similar terms to each of the following terms: 'hamlet', 'cauldron', 'nature', 'spirit', 'general', and 'prythee'

In [35]:
# List of words to find similar terms for
words_to_compare = ['hamlet', 'cauldron', 'nature', 'spirit', 'general', 'prythee']
model_names = ["CBOW", "Skip-gram", "GloVe"]

def compare_models(models, model_names):
    """Finds and prints the most similar words"""
    for model, name in zip(models, model_names):
        print(f"\nMost similar words in {name} model:")
        for word in words_to_compare:
            try:
                if hasattr(model, 'wv'):    # For Word2Vec models
                    similar_terms = model.wv.most_similar(word, topn=5)
                else:                       # For GloVe model
                    similar_terms = model.most_similar(word, topn=5)

                print(f"\nMost similar to '{word}':")
                for similar_word, similarity_score in similar_terms:
                    print(f"{similar_word}: {similarity_score:.4f}")
            except KeyError:
                print(f"'{word}' not found in {name} vocabulary.")

compare_models([cbow_model, skipgram_model, glove_model], model_names)


Most similar words in CBOW model:

Most similar to 'hamlet':
hand: 0.9996
selfe: 0.9995
heere: 0.9995
king: 0.9995
nature: 0.9995

Most similar to 'cauldron':
doth: 0.9989
euen: 0.9989
thee: 0.9989
loue: 0.9988
giue: 0.9988

Most similar to 'nature':
hath: 0.9999
haue: 0.9999
time: 0.9999
selfe: 0.9999
euen: 0.9998

Most similar to 'spirit':
haue: 0.9998
death: 0.9998
hand: 0.9998
hath: 0.9998
selfe: 0.9998

Most similar to 'general':
roman: 0.9885
braine: 0.9884
knowes: 0.9882
power: 0.9882
england: 0.9881

Most similar to 'prythee':
spirit: 0.9980
loue: 0.9980
speech: 0.9980
sonne: 0.9980
day: 0.9980

Most similar words in Skip-gram model:

Most similar to 'hamlet':
laertes: 0.9928
queene: 0.9920
polonius: 0.9914
macb: 0.9905
ham: 0.9905

Most similar to 'cauldron':
distemper: 0.9986
milke: 0.9984
ouer: 0.9983
scale: 0.9982
giuing: 0.9982

Most similar to 'nature':
magicke: 0.9952
heauie: 0.9952
mischance: 0.9951
distemper: 0.9951
vertue: 0.9950

Most similar to 'spirit':
reuenge: 0

## Commenting on how well each model captured the meaning of the word, and if there are multiple meanings, which meaning was given

### <span style='color:blue'>**'hamlet':** cbow_model and skipgram_model did a great job  linking 'hamlet' to characters from Shakespeare's play, keeping the literary essence alive. In contrast, glove_model seems to think 'hamlet' is more about places, missing the character's importance.</span>

### <span style='color:blue'>**'cauldron':** cbow_model gave a mixed bag of unrelated words, while skipgram_model throwed out terms that make you think about what's happening with a cauldron, like 'distemper.' glove_model sticked to physical items related to cauldrons, which makes more sense.</span>

### <span style='color:blue'>**'nature':** cbow_model dived into deeper, philosophical ideas, linking 'nature' with different states of being. skip_gram model leaned into the mystical side, while glove_model kept it grounded, connecting 'nature' to real-world concepts.</span>

### <span style='color:blue'>**'spirit':** cbow_model tied 'spirit' to deep feelings and existential themes. skipgram_model throwed in words like 'revenge' and 'wound.' glove_model, on the other hand, focused on emotional aspects like 'passion' and 'faith.'</span>

### <span style='color:blue'>**'general':** cbow_model connects 'general' with military and political themes. skipgram_model backed this up with hierarchy-related terms. glove_model, however, went broader, which can feel out of place for Shakespeare's time.</span>

### <span style='color:blue'>**'prythee':** cbow_model got 'prythee' right by linking it to terms of asking or pleading. skipgram_model did okay but missed the essence of the request. glove_model didn't even recognize 'prythee,' showing it struggled with older words.</span>

## Comparing the three models by finding the cosine similarity between the following pairs of terms: ('brutus', 'murder'), ('lady macbeth', 'queen gertrude'), ('fortinbras', 'norway'), ('rome', 'norway'), ('ghost', 'spirit'), ('macbeth', 'hamlet')

In [46]:
def cosine_similarity(model, word_1, word_2):
    """Calculate cosine similarity between two words, based on a model provided."""
    try:
        # Accessing word vectors using the appropriate method
        if hasattr(model, 'wv'): # For Word2Vec models
            vec1 = model.wv[word_1]
            vec2 = model.wv[word_2] 
        else: # For GloVe model
            vec1 = model.get_vector(word_1)
            vec2 = model.get_vector(word_2)
        
        # Calculating cosine similarity
        similarity = 1 - cosine(vec1, vec2)
        return similarity
    except KeyError:
        # In case the words are not found in the model
        print(f"One of the words '{word_1}' or '{word_2}' is not in the vocabulary.")
        return None

In [47]:
# Defining the word pairs to compare
word_pairs = [
    ('brutus', 'murder'),
    ('lady macbeth', 'queen gertrude'),
    ('fortinbras', 'norway'),
    ('rome', 'norway'),
    ('ghost', 'spirit'),
    ('macbeth', 'hamlet')
]

In [49]:
for model, name in zip([cbow_model, skipgram_model, glove_model], model_names):
    # Comparing the three models to find similarities between word pairs given 
    print(f"\nCosine Similarities for {name} model:\n")
    
    for word_1, word_2 in word_pairs:
        similarity = cosine_similarity(model, word_1, word_2)
        if similarity is not None:
            print(f"Similarity between '{word_1}' and '{word_2}': {similarity:.4f}")
        else:
            print(f"One or both words not found in {name} vocabulary.")


Cosine Similarities for CBOW model:

Similarity between 'brutus' and 'murder': 0.9792
One of the words 'lady macbeth' or 'queen gertrude' is not in the vocabulary.
One or both words not found in CBOW vocabulary.
Similarity between 'fortinbras' and 'norway': 0.9982
Similarity between 'rome' and 'norway': 0.9981
Similarity between 'ghost' and 'spirit': 0.9989
Similarity between 'macbeth' and 'hamlet': 0.9989

Cosine Similarities for Skip-gram model:

Similarity between 'brutus' and 'murder': 0.9719
One of the words 'lady macbeth' or 'queen gertrude' is not in the vocabulary.
One or both words not found in Skip-gram vocabulary.
Similarity between 'fortinbras' and 'norway': 0.9978
Similarity between 'rome' and 'norway': 0.9929
Similarity between 'ghost' and 'spirit': 0.9909
Similarity between 'macbeth' and 'hamlet': 0.9238

Cosine Similarities for GloVe model:

Similarity between 'brutus' and 'murder': 0.0736
One of the words 'lady macbeth' or 'queen gertrude' is not in the vocabulary.
On

## Commenting on how well each model captured the similarity between these terms, especially considering the data that each was trained on

### <span style='color:blue'>cbow_model and skipgram_model excelled in capturing the relationships between words in the context of Shakespeare’s plays, likely because they learned directly from the text itself. glove_model, while powerful for general semantic relationships, didn’t quite hit the mark for these specific literary terms, showing the importance of training context in word embeddings.</span>

## Comparing the three models by finding the 5 most similar terms to each of the following word vectors obtained via linear combination: 'denmark' + 'queen', 'scotland' + 'army' + 'general', 'father' - 'man' + 'woman', 'mother' - 'woman' + 'man'

In [60]:
# Defining the word pairs for linear combinations
word_combinations = [
    ('denmark', 'queen'),
    ('scotland', 'army', 'general'),
    ('father', 'man', 'woman'),
    ('mother', 'woman', 'man')
]

def find_similar_terms(models, model_names, word_combinations):
    """Finds most similar words for each model based on linear combinations"""
    
    for model, name in zip(models, model_names):
        print(f"\nMost similar words in {name} model:\n")

        for index, combo in enumerate(word_combinations):
            try:
                if hasattr(model, 'wv'):  # For Word2Vec models
                    vec1 = model.wv[combo[0]]
                    vec2 = model.wv[combo[1]]
                    if len(combo) == 3:
                        vec3 = model.wv[combo[2]]
                    else:
                        vec3 = 0  # No third vector, set to 0

                else:  # For GloVe model
                    vec1 = model.get_vector(combo[0])
                    vec2 = model.get_vector(combo[1])
                    if len(combo) == 3:
                        vec3 = model.get_vector(combo[2])
                    else:
                        vec3 = 0  # No third vector, set to 0

               # Computing the resulting vector
                if index < 2:
                    vector = sum((vec1 + vec2), vec3)
                else:
                    vector = vec1 - vec2 + vec3

                # Finding the most similar words
                if hasattr(model, 'wv'):  # For Word2Vec models
                    similar_terms = model.wv.similar_by_vector(vector, topn=5)
                else:  # For GloVe model
                    similar_terms = model.similar_by_vector(vector, topn=5)

                print(f"\nSimilar terms for {combo}:")
                for term, similarity in similar_terms:
                    print(f"{term}: {similarity:.4f}")

            except KeyError as e:
                print(f"One of the words {combo} not found in {name} vocabulary: {e}")
            except Exception as e:
                print(f"An error occurred for combo {combo}: {e}")

find_similar_terms([cbow_model, skipgram_model, glove_model], model_names, word_combinations)


Most similar words in CBOW model:

An error occurred for combo ('denmark', 'queen'): invalid index to scalar variable.

Similar terms for ('scotland', 'army', 'general'):
clau: 0.3076
slept: 0.1897
sparkes: 0.1887
couch: 0.1857
drawne: 0.1819

Similar terms for ('father', 'man', 'woman'):
father: 0.9999
thy: 0.9998
haue: 0.9997
vpon: 0.9997
death: 0.9997

Similar terms for ('mother', 'woman', 'man'):
mother: 0.9991
thee: 0.9985
god: 0.9985
night: 0.9985
brother: 0.9985

Most similar words in Skip-gram model:

An error occurred for combo ('denmark', 'queen'): invalid index to scalar variable.

Similar terms for ('scotland', 'army', 'general'):
clau: 0.3119
pind: 0.1618
finis: 0.1440
puh: 0.1414
preferre: 0.1345

Similar terms for ('father', 'man', 'woman'):
father: 0.9959
mother: 0.9883
lost: 0.9879
wife: 0.9855
woman: 0.9832

Similar terms for ('mother', 'woman', 'man'):
mother: 0.9960
wife: 0.9891
ayde: 0.9888
weepe: 0.9887
father: 0.9886

Most similar words in GloVe model:

An error

## Commenting on how well each model described the ideas behind these word vectors

### <span style='color:blue'>While each model has its strengths, they all have room for improvement, especially when it comes to handling tricky word pairings. Their successes show they can capture family relationships, but they need to work on understanding deeper or more nuanced ideas. Evaluating each combo results, we have:</span>

### <span style='color:blue'>Insights for cbow_model:</span>
* <span style='color:blue'>Error for ('denmark' + 'queen'): The model couldn't process this combo correctly, probably because it couldn't figure out how to combine those words properly.</span>
* <span style='color:blue'>('scotland' + 'army' + 'general'): The words it returned feel a bit random, not really capturing the military essence that we were hoping for.</span>
* <span style='color:blue'>('father' - 'man' + 'woman'): This one worked pretty well! It really focused on "father," showing it gets the male side of family relationships.</span>
* <span style='color:blue'>('mother' - 'woman' + 'man'): Here, it did well too, strongly linking to "mother," which fits with the nurturing role traditionally associated with women.</span>

### <span style='color:blue'>Insights for skipgram_model:</span>
* <span style='color:blue'>Error for ('denmark' + 'queen'): Same story here; it couldn’t process this combo correctly.</span>
* <span style='color:blue'>('scotland' + 'army' + 'general'): The results were still a bit off, suggesting it struggled to grasp the military context like the cbow_model.</span>
* <span style='color:blue'>('father' - 'man' + 'woman'): This model fared better with family terms, emphasizing "father" and also including "mother" and "wife," showing it has a decent grasp on gender roles.</span>
* <span style='color:blue'>('mother' - 'woman' + 'man'): Here, it did well too, strongly linking to "mother," which fits with the nurturing role traditionally associated with women.</span>

### <span style='color:blue'>Insights for glove_model:</span>
* <span style='color:blue'>Error for ('denmark' + 'queen'): The model couldn't process this combo correctly as in previous models.</span>
* <span style='color:blue'>('scotland' + 'army' + 'general'): TThis model did a bit better with military terms, offering up words like "unit" and "commander," though they still felt a bit generic.</span>
* <span style='color:blue'>('father' - 'man' + 'woman'): It gave relevant words like "mother" and "daughter," showing it understands family relationships, but it leans towards traditional roles.</span>
* <span style='color:blue'>('mother' - 'woman' + 'man'): It did a good job here, associating with "father" and "brother," showing it gets family dynamics pretty well.</span>

## Overall comments on how each model performs, describing what data you would use to train a better word embedding model to captures the meaning of Shakespearean English

### <span style='color:blue'>The cbow_model and skipgram_model showed strong potential in recognizing relationships within the context of Shakespeare’s plays, thanks to their training on relevant texts. However, both models could benefit from broader datasets that include historical and thematic context. The glove_model, while useful for general semantic relationships, fell short in understanding the specific literary nuances, highlighting the importance of context in word embeddings. By expanding their training data to include more comprehensive Shakespearean texts and related literature, these models could significantly improve their performance in capturing the richness of Shakespearean English.</span>